In [1]:
import os, sys
from keras import backend as K

%matplotlib inline

Using Theano backend.
Using gpu device 0: GeForce GTX 750 Ti (CNMeM is disabled, cuDNN 5103)
/home/manpun/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


## First get the training, validation and sample data in order.

In [2]:
homeDir = os.getcwd()

In [3]:
dataDir = homeDir + "/data/"
#dataDir = homeDir + "/data/sample/"
train_path = dataDir + "train/"
valid_path = dataDir + "valid/"
model_path = homeDir + "/data/models/"
test_path = dataDir + "test"

In [4]:
from utils import *
from Vgg16 import Vgg16
from vgg16bn import Vgg16BN

### Getting the data ready for processing.

In [ ]:
#trn_data = get_data(batches)
#val_data = get_data(val_batches)

In [ ]:
batches

## Get the vgg16 model and initialize it

In [5]:
vgg = Vgg16()

In [6]:
#Set constants. You can experiment with no_of_epochs to improve the model. You can reduce the batch_size 
#depending on the memory contraints of gpu
batch_size=64
no_of_epochs=3

In [17]:
# get_batches function transforms images into arrays and the gets them in batches.

batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
vgg.model.summary()

## Finetune 
Now I will remove the final dense layer for classifying in to 1000 categories and add a new layer for only cats and dogs.

In [8]:
vgg.model.pop() #Remove the final layer
for layer in vgg.model.layers:
    layer.trainable=False # Set all other layers to untrainable

In [9]:
vgg.model.add(Dense(2, activation='softmax')) 
#Adding a new dense layer wiht only 2 outputs and softmax acitvation as it is the output layer

In [ ]:
# Using different optimizers 
opt = RMSprop(lr=0.01)


In [10]:
vgg.compile()
#vgg.model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
23000/23000 [==============================] - 609s - loss: 0.1240 - acc: 0.9684 - val_loss: 0.0762 - val_acc: 0.9800


In [ ]:
vgg.model.save_weights(model_path+'finetune_last_layer.h5')

Now let us train all the dense layers and see the results

In [18]:
layers= vgg.model.layers

In [19]:
dense_layer_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense]

In [20]:
dense_layer_idx

[33, 35, 37]

In [21]:
for l in dense_layer_idx:
    layers[l].trainable=True

In [22]:
layers[34].trainable

False

In [24]:
#K.set_value(opt.lr, 0.01)
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
23000/23000 [==============================] - 759s - loss: 0.0927 - acc: 0.9783 - val_loss: 0.0865 - val_acc: 0.9795


In [25]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
23000/23000 [==============================] - 975s - loss: 0.0930 - acc: 0.9795 - val_loss: 0.0890 - val_acc: 0.9810


In [27]:
vgg.model.save_weights(model_path+'finetune_allDenseLayers.h5')

### Predictions here 

In [11]:
results_path = dataDir + "/results"

In [28]:
batchnames, preds = vgg.test(test_path, batch_size = batch_size*2)

Found 12500 images belonging to 1 classes.


NameError: name 'batchenames' is not defined

In [29]:
filenames = batchnames.filenames

In [ ]:
#Save our test results arrays so we can use them again later
save_array(results_path + 'test_preds.dat', preds)
save_array(results_path + 'filenames.dat', filenames)
preds = load_array(results_path + 'test_preds.dat')
filenames = load_array(results_path + 'filenames.dat')

In [30]:
isdog = preds[:,1]
isdog = isdog.clip(min=0.025, max=0.975)

In [31]:
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
subm = np.stack([ids,isdog], axis=1)
subm[:5]

array([[  9.2920e+03,   2.5000e-02],
       [  1.2026e+04,   1.2684e-01],
       [  9.6880e+03,   2.5000e-02],
       [  4.3920e+03,   2.5000e-02],
       [  7.7900e+02,   9.7500e-01]])

In [32]:
%cd $dataDir
submission_file_name = 'submission_ft_denseLayers.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')

/home/ubuntu/KaggleCompetitions/cats_dogs_redux/data


In [33]:
from IPython.display import FileLink
%cd ../
FileLink('data/'+submission_file_name)

/home/ubuntu/KaggleCompetitions/cats_dogs_redux


/home/ubuntu/KaggleCompetitions/cats_dogs_redux/data/submission_ft_denseLayers.csv

## It lloks like we are underfitting the model.
Let us modify the vgg model for our purpose

In [ ]:
model = vgg_ft(2)

In [ ]:
model.load_weights(model_path + 'finetune_last_layer.h5')

In [34]:
model = vgg.model

In [ ]:
model.load_weights(model_path + 'finetune_last_layer.h5')

In [35]:
layers = model.layers

In [36]:
layers[-1].output_shape

(None, 2)

In [37]:
lastConvIdx = [index for index,layer in enumerate(layers) if type(layer) is Convolution2D][-1]

Creating a seperate model for conv and fc layers

In [63]:
convLayers = layers[:lastConvIdx+1]
convModel = Sequential(convLayers)
fcLayers = layers[lastConvIdx+1:]

In [39]:
batches = get_batches(train_path, shuffle=False, batch_size = batch_size)
val_batches = get_batches(valid_path, shuffle=False, batch_size=batch_size)
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


SAving the oupiut of convolution block so that the processing is not needed every time we fit .

In [40]:
batches.nb_sample

23000

In [41]:
train_features = convModel.predict_generator(batches,batches.nb_sample)
val_features = convModel.predict_generator(val_batches, val_batches.nb_sample)

In [42]:
save_array(model_path + 'train_convlayer_features.bc', train_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [43]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [44]:
trn_features.shape

(23000, 512, 14, 14)

In [50]:
def new_weights(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]
opt = RMSprop(lr=0.00001, rho=0.7)
def get_fc_model(p):
    model = Sequential([
        MaxPooling2D(input_shape = convLayers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096,activation="relu"),
        Dropout(p),
        #BatchNormalization(),
        Dense(4096, activation='relu'),
        Dropout(p),
        #BatchNormalization(),
        Dense(2, activation='softmax')
    ])
    for l1,l2 in zip(model.layers, fcLayers): l1.set_weights(new_weights(l2,0.5,p))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [46]:
fcLayers[-1].get_weights()[1].shape

(2,)

In [55]:
fc_model = get_fc_model(0.6)

In [56]:
fc_model.fit(trn_features, trn_labels, nb_epoch=5, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/5
23000/23000 [==============================] - 34s - loss: 0.0874 - acc: 0.9846 - val_loss: 0.0990 - val_acc: 0.9820
Epoch 2/5
23000/23000 [==============================] - 34s - loss: 0.0514 - acc: 0.9901 - val_loss: 0.1034 - val_acc: 0.9830
Epoch 3/5
23000/23000 [==============================] - 34s - loss: 0.0356 - acc: 0.9940 - val_loss: 0.1131 - val_acc: 0.9820
Epoch 4/5
23000/23000 [==============================] - 34s - loss: 0.0219 - acc: 0.9963 - val_loss: 0.1242 - val_acc: 0.9830
Epoch 5/5
23000/23000 [==============================] - 34s - loss: 0.0163 - acc: 0.9975 - val_loss: 0.1242 - val_acc: 0.9835


In [ ]:
def proc_wgts(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]

In [ ]:
fc_model.fit(trn_features, trn_labels, nb_epoch=5, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

In [ ]:
#conv features for test set
test_features = convModel.predict_generator(test_batches,test_batches.nb_sample)
save_array(model_path + 'test_convlayer_features.bc', test_features)
test_features = load_array(model_path+'test_convlayer_features.bc')
# predicting the results
preds = fc_model.predict_generator(test_features, test_batches.nb_sample)

In [64]:
for layer in convModel.layers: layer.trainable = False
# Look how easy it is to connect two models together!
convModel.add(fc_model)

In [65]:
opt = RMSprop(lr=0.00001, rho=0.7)
convModel.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [66]:
convModel.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/1
23000/23000 [==============================] - 609s - loss: 0.0316 - acc: 0.9943 - val_loss: 0.1741 - val_acc: 0.9795


In [68]:
convModel.save_weights(model_path + 'dropout6.h5')

In [69]:
test_path = dataDir + "/test"
test_batches = get_batches(test_path,shuffle=False, batch_size = batch_size)

Found 12500 images belonging to 1 classes.


In [71]:
preds = convModel.predict_generator(test_batches, test_batches.nb_sample)

In [72]:
filenames = test_batches.filenames

In [73]:
isdog = preds[:,1]
isdog = isdog.clip(min=0.025, max=0.975)
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
subm = np.stack([ids,isdog], axis=1)
subm[:5]
%cd $dataDir
submission_file_name = 'submission_ft_denseLayers_droupout6.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')
from IPython.display import FileLink
%cd ../
FileLink('data/'+submission_file_name)

/home/ubuntu/KaggleCompetitions/cats_dogs_redux/data
/home/ubuntu/KaggleCompetitions/cats_dogs_redux


/home/ubuntu/KaggleCompetitions/cats_dogs_redux/data/submission_ft_denseLayers_droupout6.csv

## Batch Normalization

Currently it looks like we are overfitting. Hence we shall use batchnormalization

In [5]:
vgg = Vgg16BN()

/home/manpun/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:621: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [ ]:
#Set constants. You can experiment with no_of_epochs to improve the model. You can reduce the batch_size 
#depending on the memory contraints of gpu
batch_size=64
no_of_epochs=3

In [ ]:
# get_batches function transforms images into arrays and the gets them in batches.

batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size)

In [ ]:
vgg.model.pop() #Remove the final layer
for layer in vgg.model.layers:
    layer.trainable=False # Set all other layers to untrainable

In [ ]:
vgg.model.add(Dense(2, activation='softmax')) 
#Adding a new dense layer wiht only 2 outputs and softmax acitvation as it is the output layer

In [ ]:
vgg.compile()
#vgg.model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
vgg.fit(batches, val_batches, nb_epoch=1)

In [ ]:
model = vgg.model

In [ ]:
layers = model.layers

In [ ]:
lastConvIdx = [index for index,layer in enumerate(layers) if type(layer) is Convolution2D][-1]

In [ ]:
convLayers = layers[:lastConvIdx+1]
convModel = Sequential(convLayers)
fcLayers = layers[lastConvIdx+1:]

In [ ]:
batches = get_batches(train_path, shuffle=False, batch_size = batch_size)
val_batches = get_batches(valid_path, shuffle=False, batch_size=batch_size)
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [ ]:
train_features = convModel.predict_generator(batches,batches.nb_sample)
val_features = convModel.predict_generator(val_batches, val_batches.nb_sample)

In [ ]:
save_array(model_path + 'train_convlayer_features.bc', train_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [ ]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [ ]:
def new_weights(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]
opt = RMSprop(lr=0.00001, rho=0.7)
def get_fc_model(p):
    model = Sequential([
        MaxPooling2D(input_shape = convLayers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096,activation="relu"),
        Dropout(p),
        BatchNormalization(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(2, activation='softmax')
    ])
    for l1,l2 in zip(model.layers, fcLayers): l1.set_weights(new_weights(l2,0.5,p))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
fc_model = get_fc_model(0.6)

In [ ]:
fc_model.fit(trn_features, trn_labels, nb_epoch=5, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

In [ ]:
for layer in convModel.layers: layer.trainable = False
# Look how easy it is to connect two models together!
convModel.add(fc_model)

In [ ]:
opt = RMSprop(lr=0.00001, rho=0.7)
convModel.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
convModel.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

In [ ]:
test_path = dataDir + "/test"
test_batches = get_batches(test_path,shuffle=False, batch_size = batch_size)

In [ ]:
preds = convModel.predict_generator(test_batches, test_batches.nb_sample)

In [ ]:
filenames = test_batches.filenames
isdog = preds[:,1]
isdog = isdog.clip(min=0.025, max=0.975)
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
subm = np.stack([ids,isdog], axis=1)
subm[:5]
%cd $dataDir
submission_file_name = 'submission_ft_denseLayers_droupout6.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')
from IPython.display import FileLink
%cd ../
FileLink('data/'+submission_file_name)

## data augmentation
trying out data augmentation to reduce the overfitting.

In [ ]:
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, 
       height_shift_range=0.1, width_zoom_range=0.2, shear_range=0.15, zoom_range=0.1, 
       channel_shift_range=10., horizontal_flip=True)

In [ ]:
batches = get_batches(path+'train', gen, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

In [ ]:
fc_model = get_fc_model()

In [ ]:
for layer in conv_model.layers: layer.trainable = False
# Look how easy it is to connect two models together!
conv_model.add(fc_model)

In [ ]:
conv_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
conv_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=8, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)